In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display as display
import pandas as pd

In [2]:
us_dir = './UrbanSound8K/'
audio_dir = us_dir+'/audio'
ds = pd.read_csv(us_dir+'/metadata/UrbanSound8K.csv')
ds.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [12]:
train_idx = ds.index[(ds['fold'] == 1) | (ds['fold']==2)].tolist()
val_idx = ds.index[(ds['fold'] == 3) | (ds['fold']==4)].tolist()
train_set = ds.iloc[train_idx]
val_set = ds.iloc[val_idx]
train_set.tail()

,slice_file_name,fsID,start,end,salience,fold,classID,class
8676,99180-9-0-2.wav,99180,1.0,5.0,1,1,9,street_music
8677,99180-9-0-36.wav,99180,18.0,22.0,1,1,9,street_music
8678,99180-9-0-48.wav,99180,24.0,28.0,1,1,9,street_music
8679,99180-9-0-49.wav,99180,24.5,28.5,1,1,9,street_music
8680,99180-9-0-7.wav,99180,3.5,7.5,1,1,9,street_music


In [47]:
def create_data(df):
    X_data = []
    for _,sl in df[:10].iterrows():
        y, sr = librosa.load(audio_dir+'/fold{:d}/{}'.format(sl['fold'],sl['slice_file_name']))
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        mfcc_delta = librosa.feature.delta(mfcc)
        X_data.append(mfcc_delta.ravel())
    X_data = np.vstack(X_data)
    y_data = df['classID'].to_numpy()
    return X_data,y_data

In [48]:
X, y = create_data(train_set)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3460 and the array at index 7 has size 700

In [ ]:
y

In [ ]:
sl = ds.iloc[10]
y, sr = librosa.load(audio_dir+'/fold{:d}/{}'.format(sl['fold'],sl['slice_file_name']))

In [ ]:
# plot wave track
plt.subplot(2, 1, 1)
display.waveplot(y, sr=sr)

In [ ]:
# plot spectrogram
plt.subplot(2, 1, 2)
D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
librosa.display.specshow(D, y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')

In [ ]:
# calculate mfcc and mfcc-delta
mfcc = librosa.feature.mfcc(y=y, sr=sr)
mfcc_delta = librosa.feature.delta(mfcc)
plt.subplot(2, 1, 1)
librosa.display.specshow(mfcc)
plt.title('MFCC')
plt.colorbar()
plt.subplot(2, 1, 2)
librosa.display.specshow(mfcc_delta)
plt.title(r'MFCC-$\Delta$')
plt.colorbar()